## Analizador Léxico
Su función es analizar una secuencia de caracteres para convertirla en una secuencia de tokens

In [ ]:
from enum import Enum

# Definición de tipos de token

class TipoToken(Enum):
    SI = 1
    ENTONCES = 2
    PALABRA = 3
    NUMERO = 4
    CC = 5
    KM = 6

In [ ]:
# Función para el analizador léxico

def analizador_lexico(entrada: str) -> list:
    tokens = entrada.lower().split()
    lexemas = []

    for token in tokens:
        if token == 'si':
            lexemas.append({"valor": token, "tipo": TipoToken.SI})
        elif token == 'entonces':
            lexemas.append({"valor": token, "tipo": TipoToken.ENTONCES})
        elif token.isdigit():
            lexemas.append({"valor": token, "tipo": TipoToken.NUMERO})
        elif token == 'cc':
            lexemas.append({"valor": token, "tipo": TipoToken.CC})
        elif token == 'km':
            lexemas.append({"valor": token, "tipo": TipoToken.KM})
        else:
            lexemas.append({"valor": token, "tipo": TipoToken.PALABRA})

    return lexemas

## Analizador sintáctico
Analiza la estructura sintáctica del código fuente para determinar si está correctamente

In [ ]:
# Definición de estados del analizador sintáctico

class EstadosAnalizador(Enum):
    INICIO = 1
    PRECEDENTE = 2
    CONDICION = 3
    CONSECUENTE = 4
    ATOMO = 5
    ERROR = 6
    FIN = 7

In [ ]:
# Función para el analizador sintáctico

def analizador_sintactico(lexemas: list, reglas_simbolos: list) -> EstadosAnalizador:
    estado_actual = EstadosAnalizador.INICIO

    for lexema in lexemas:
        tipo = lexema["tipo"]
        valor = lexema["valor"]

        if estado_actual == EstadosAnalizador.INICIO:
            if tipo == TipoToken.SI:
                estado_actual = EstadosAnalizador.PRECEDENTE
            else:
                estado_actual = EstadosAnalizador.ERROR
        
        elif estado_actual == EstadosAnalizador.PRECEDENTE:
            if tipo == TipoToken.PALABRA:
                estado_actual = EstadosAnalizador.CONDICION
            else:
                estado_actual = EstadosAnalizador.ERROR
        
        elif estado_actual == EstadosAnalizador.CONDICION:
            if tipo == TipoToken.NUMERO or tipo == TipoToken.PALABRA or tipo == TipoToken.CC or tipo == TipoToken.KM:
                estado_actual = EstadosAnalizador.ATOMO
            else:
                estado_actual = EstadosAnalizador.ERROR
        
        elif estado_actual == EstadosAnalizador.ATOMO:
            if tipo == TipoToken.ENTONCES:
                estado_actual = EstadosAnalizador.CONSECUENTE
            else:
                estado_actual = EstadosAnalizador.ERROR
        
        elif estado_actual == EstadosAnalizador.CONSECUENTE:
            if tipo == TipoToken.PALABRA:
                estado_actual = EstadosAnalizador.FIN
            else:
                estado_actual = EstadosAnalizador.ERROR

    return estado_actual

## Analizador semántico
Comprueba reglas adicionales que no son posibles de verificar en la etapa léxica o sintáctica

In [ ]:
# Función para el analizador semántico

def analizador_semantico(lexemas: list) -> bool:
    tiene_si = False
    tiene_entonces = False

    for lexema in lexemas:
        if lexema["tipo"] == TipoToken.SI:
            tiene_si = True
        elif lexema["tipo"] == TipoToken.ENTONCES:
            tiene_entonces = True

    return tiene_si and tiene_entonces

## Lector de reglas

In [ ]:
# Función para leer las reglas desde un archivo

def leer_reglas(nombre_archivo: str) -> list:
    with open(nombre_archivo, 'r') as file:
        reglas = file.readlines()
    return [regla.strip() for regla in reglas if regla.strip()]

In [ ]:
# Leer las reglas desde el archivo

nombre_archivo = 'reglas.txt'
reglas = leer_reglas(nombre_archivo)

## Convertir reglas a símbolo

In [ ]:
# Función para convertir las reglas en símbolos para el analizador sintáctico

def convertir_a_simbolos(reglas: list, analizador_lexico) -> list:
    reglas_simbolos = []
    for regla in reglas:
        lexemas = analizador_lexico(regla)
        reglas_simbolos.append(lexemas)
    return reglas_simbolos

In [ ]:
# Convertir las reglas en símbolos

reglas_simbolos = convertir_a_simbolos(reglas, analizador_lexico)

## Generación de tabla

In [ ]:
# Función para construir la tabla de reglas para mostrar
def construir_tabla_reglas(reglas_simbolos: list) -> dict:
    tabla_reglas = {}

    for i, regla_simbolos in enumerate(reglas_simbolos, start=1):
        tabla_reglas[f"Regla {i}"] = regla_simbolos

    return tabla_reglas

In [ ]:
# Construir la tabla de reglas para mostrar

tabla_reglas = construir_tabla_reglas(reglas_simbolos)

## Ejemplo de aplicación

In [ ]:
# Ejemplos de frases para probar el analizador

frases = [
    "Si la motocicleta tiene más de 140 cc entonces es considerada de alto cilindraje",
    "Si la motocicleta tiene menos de 3 años de antigüedad entonces es nueva",
    "Si la motocicleta tiene menos de 500 km entonces es casi nueva",
    "Si la motocicleta es deportiva y tiene más de 500 cc entonces es de alto rendimiento",
    "Si la motocicleta es chopper y tiene más de 1000 cc entonces es de alto cilindraje",
    "La motocicleta tiene menos de 150 cc entonces es considerada de bajo cilindraje",
    "Si la motocicleta tiene al menos 4 años de antigüedad entonces es usada",
]

## Ejecución de analizador

In [ ]:
# Ejecución del analizador para las frases

for frase in frases:
    lexemas = analizador_lexico(frase)
    resultado_sintactico = analizador_sintactico(lexemas, reglas_simbolos)
    es_valido = analizador_semantico(lexemas)

    print(frase)
    if resultado_sintactico == EstadosAnalizador.FIN and es_valido:
        print("La frase es válida.")
    else:
        print("La frase no es válida.")

    print("Resultado sintáctico:", resultado_sintactico)
    print("Resultado semántico:", es_valido)
    print()

# Mostrar tabla de reglas
print("Tabla de Reglas:")
for nombre_regla, regla_simbolos in tabla_reglas.items():
    print(nombre_regla)
    for simbolo in regla_simbolos:
        print(simbolo)
    print()